In [8]:
# %pip install tensorflow[and-cuda]

In [9]:

import numpy as np
import os
import pandas as pd
from tensorflow.keras.utils import image_dataset_from_directory
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras

from tensorflow.keras.layers.experimental import preprocessing
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Model



In [10]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
import matplotlib.pyplot as plt


In [11]:
def load_cnn():
  cnn = VGG16(weights='imagenet', include_top=False)
  model = Sequential()

  # Add the VGG16 convolutional base
  model.add(cnn)

  # Add a Flatten layer to transition from convolutional to dense layers
  model.add(Flatten())
  return model



In [12]:
model = load_cnn()
model.summary()


58889256/58889256 [==============================] - 8s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, None, None, 512)   14714688  
                                                                 
 flatten (Flatten)           (None, None)              0         
                                                                 
Total params: 14714688 (56.13 MB)
Trainable params: 14714688 (56.13 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
def get_datasets():
  training_dataset, test_dataset = image_dataset_from_directory(
    '/content/drive/Shareddrives/AADT/images',
     labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(224, 224),
    shuffle=True,
    seed=7676,
    validation_split=0.2,
    subset="both",
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
  )
  return training_dataset, test_dataset





In [14]:
import re


def getId(pt):
  match = re.search(r'i([^_/]+)_([^\.]+)\.png$', pt)
  if match:
      p1 = match.group(1)
      p2 = match.group(2)
      result = f"{p1}_{p2}"
  else:
      result = None
  return result

In [15]:
# import os
# import shutil

# # Define source and destination directories
# source_dir = '/content/drive/Shareddrives/AADT/images/images'
# dest_dir = '/content/drive/Shareddrives/AADT/imgs'

# # Ensure the destination directory exists
# os.makedirs(dest_dir, exist_ok=True)
# print(len(os.listdir(source_dir)))
# # Iterate over all files in the source directory
# for filename in os.listdir(source_dir):
#     print(filename)
#     if filename.endswith('.png'):
#         # Extract the p1 part of the filename
#         p1 = getId(filename)  # Assuming filename is of the form i<p1>.png

#         # Define the new directory path
#         new_dir = os.path.join(dest_dir, p1)

#         # Create the new directory if it doesn't exist
#         os.makedirs(new_dir, exist_ok=True)

#         # Define the full file paths
#         src_file = os.path.join(source_dir, filename)
#         dest_file = os.path.join(new_dir, filename)

#         # Move the file
#         shutil.move(src_file, dest_file)

In [16]:
full_count_df = pd.read_csv('full_count.csv')
classes = sorted(list(os.listdir('image_folders/')))
classes = [c.replace("_", "@") for c in classes]
classes


['000001@01',
 '000001@08',
 '000001@13',
 '000001@16',
 '000001@22',
 '000001@25',
 '000001@31',
 '000001@42',
 '000001@45',
 '000001@54',
 '000002@01',
 '000002@08',
 '000002@13',
 '000002@15',
 '000002@16',
 '000002@22',
 '000002@25',
 '000002@31',
 '000002@42',
 '000002@45',
 '000002@54',
 '000003@08',
 '000003@13',
 '000003@15',
 '000003@16',
 '000003@22',
 '000003@25',
 '000003@31',
 '000003@45',
 '000003@47',
 '000003@54',
 '000004@08',
 '000004@15',
 '000004@16',
 '000004@22',
 '000004@31',
 '000004@42',
 '000004@45',
 '000004@47',
 '000004@48',
 '000004@54',
 '000005@08',
 '000005@16',
 '000005@25',
 '000005@31',
 '000005@42',
 '000005@45',
 '000005@47',
 '000006@08',
 '000006@15',
 '000006@16',
 '000006@25',
 '000006@31',
 '000006@45',
 '000006@46',
 '000006@47',
 '000006@48',
 '000006@54',
 '000007@01',
 '000007@08',
 '000007@16',
 '000007@22',
 '000007@25',
 '000007@31',
 '000007@39',
 '000007@42',
 '000007@45',
 '000007@47',
 '000007@48',
 '000007@54',
 '000008@08',
 '0000

In [17]:
def get_average_by_id( id):
    id = classes[id]
    return df.loc[full_count_df['id'] == id, 'average'].values[0]

In [18]:
def get_complete_dataset():
  dataset = image_dataset_from_directory(
      'image_folders',
      labels="inferred",
      label_mode="int",
      class_names=None,
      color_mode="rgb",
      batch_size=32,
      image_size=(224, 224),
      shuffle=True,
      seed=7676,
      validation_split=0,
      interpolation="bilinear",
      follow_links=False,
      crop_to_aspect_ratio=False,
  )
  gen = ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True,

  )
  # Define the augmentation layer
  data_augmentation = tf.keras.Sequential([
      preprocessing.RandomFlip("horizontal_and_vertical"),
      preprocessing.RandomRotation(
        (-1.57079633 , 1.57079633)
    )
  ])

  # Apply the augmentation to the dataset
  def augment(images, labels):
      images = data_augmentation(images)
      # labels = classes[labels]
      return images, labels

  augmented_dataset = dataset.map(augment)

  return augmented_dataset

In [19]:
# os.listdir('/content/drive/Shareddrives/AADT/image_folders/000001_13')

In [20]:
dataset = get_complete_dataset()
cnn= load_cnn()

features=[]
labels= []
for step in range(4):
  print("======", step, "======")
  for img_batch, label_batch in dataset:

      labels.append(label_batch)
      x = preprocess_input(img_batch)
      p = cnn.predict(x)
      # shape = p.shape

      # # Calculate M
      # M = shape[1] * shape[2] * shape[3]
      features.append(p)
      print("step")



labels = tf.concat(labels, axis=0)
features = tf.concat(features, axis=0)
labels
files = labels

Found 4820 files belonging to 4820 classes.
====== 0 ======
1/1 [==============================] - 3s 3s/step
step
1/1 [==============================] - 3s 3s/step
step
1/1 [==============================] - 3s 3s/step
step
1/1 [==============================] - 2s 2s/step
step
1/1 [==============================] - 2s 2s/step
step
1/1 [==============================] - 2s 2s/step
step
1/1 [==============================] - 3s 3s/step
step
1/1 [==============================] - 3s 3s/step
step
1/1 [==============================] - 4s 4s/step
step
1/1 [==============================] - 5s 5s/step
step
1/1 [==============================] - 5s 5s/step
step
1/1 [==============================] - 5s 5s/step
step
1/1 [==============================] - 5s 5s/step
step
1/1 [==============================] - 5s 5s/step
step
1/1 [==============================] - 5s 5s/step
step
1/1 [==============================] - 5s 5s/step
step
1/1 [==============================] - 5s 5s/step
step
1/1 [

In [21]:
labels

<tf.Tensor: shape=(19280,), dtype=int32, numpy=array([4593, 2705,  906, ..., 4444, 4014, 4499])>

In [22]:
ids = [ classes[f] for f in files ]
ids


['0R3102@37',
 '004203@29',
 '000153@45',
 '00034B@34',
 '0W7501@37',
 '001531@36',
 '006480@36',
 '000415@25',
 '000052@31',
 '000020@45',
 '080499@51',
 '0A2501@37',
 '005031@29',
 '100118@04',
 '000280@13',
 '000506@42',
 '00Y006@50',
 '0P0013@24',
 '000800@42',
 '051604@23',
 '003180@25',
 '000289@19',
 '002202@48',
 '0A6403@37',
 '000001@01',
 '059705@23',
 '009109@48',
 '000178@48',
 '000306@19',
 '000659@49',
 '000486@47',
 '063013@15',
 '004049@25',
 '00N123@39',
 '0000P2@48',
 '000022@25',
 '049000@06',
 '039023@33',
 '000248@16',
 '000714@19',
 '000068@45',
 '027006@41',
 '020267@51',
 '000036@31',
 '000183@46',
 '000378@42',
 '100373@04',
 '001210@17',
 '000155@39',
 '000267@16',
 '000260@08',
 '003669@48',
 '005124@25',
 '000054@22',
 '000054@48',
 '000489@47',
 '034520@06',
 '100676@04',
 '080132@51',
 '035730@06',
 '050238@51',
 '000N81@39',
 '000077@48',
 '000822@42',
 '100343@04',
 '000520@01',
 '039650@06',
 '000771@39',
 '040011@51',
 '000232@16',
 '0R5802@37',
 '000N

In [23]:
df = pd.DataFrame(data=features)
str_df = pd.DataFrame(data=ids[:len(df)], columns=['id'])
df = pd.concat([str_df, df], axis=1)

df


,id,0,1,2,3,4,5,6,7,8,...,25078,25079,25080,25081,25082,25083,25084,25085,25086,25087
0,0R3102@37,12.539886,0.000000,0.0,0.0,0.000000,0.0,0.0,1.476546,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,14.415866,0.0
1,004203@29,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.289230,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,000153@45,6.865058,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,00034B@34,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,0W7501@37,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,4.508424,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19275,005182@36,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.000000,3.291706,20.477949,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
19276,069507@23,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,10.384589,0.000000,1.905236,0.0,0.0,0.0,0.0,0.0,8.314670,0.0
19277,099278@33,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,7.044128,0.0,...,0.000000,0.000000,6.711871,0.0,0.0,0.0,0.0,0.0,13.118344,0.0
19278,050048@51,3.144773,11.913834,0.0,0.0,10.343277,0.0,0.0,0.000000,0.0,...,10.353261,0.000000,17.589960,0.0,0.0,0.0,0.0,0.0,3.563257,0.0


In [24]:
df.to_csv('vgg_2.csv')

In [25]:
baseModel = load_cnn()
baseModel.trainable = False ## Not trainable weights
dense_layer_1 = Dense(64, activation='relu')
dense_layer_2 = Dense(32, activation='relu')
dense_layer_3 = Dense(16, activation='relu')
model = Sequential([
    baseModel,
    dense_layer_1,
    dense_layer_2,
    dense_layer_3,
    Dense(1)  # Output layer for regression
])

ValueError: The last dimension of the inputs to a Dense layer should be defined. Found None. Full input shape received: (None, None)